In [1]:
from langchain_community.llms import Ollama
llm = Ollama(model="deepseek-r1:1.5b")
response = llm.invoke("What is the capital of china")
print(response)

C:\Users\mohit\AppData\Local\Temp\ipykernel_4296\2443101709.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="deepseek-r1:1.5b")


<think>

</think>

The capital of China is Beijing.


In [2]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(input_variables=['country'],template="tell me the capital of this {country}")
prompt.format(country="india")

'tell me the capital of this india'

In [3]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm,prompt=prompt)
print(chain.run('china'))

C:\Users\mohit\AppData\Local\Temp\ipykernel_4296\1750887829.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm,prompt=prompt)
C:\Users\mohit\AppData\Local\Temp\ipykernel_4296\1750887829.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(chain.run('china'))


<think>

</think>

The capital of China is Beijing.


In [4]:
!pip install -q youtube-transcript-api langchain-community langchain-openai faiss-cpu tiktoken python-dotenv


In [5]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings,ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

In [6]:
pip install yt-dlp


In [7]:
pip install yt-dlp


Note: you may need to restart the kernel to use updated packages.


In [8]:
import subprocess

video_url = "https://www.youtube.com/watch?v=Gfr50f6ZBvo"

subprocess.run([
    "yt-dlp",
    "--write-auto-sub",         # Use --write-sub for manually created subtitles
    "--sub-lang", "en",
    "--skip-download",
    "--output", "%(title)s.%(ext)s",
    video_url
])


CompletedProcess(args=['yt-dlp', '--write-auto-sub', '--sub-lang', 'en', '--skip-download', '--output', '%(title)s.%(ext)s', 'https://www.youtube.com/watch?v=Gfr50f6ZBvo'], returncode=0)

In [9]:
def parse_vtt(file_path):
    lines = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip() and not line.startswith(("WEBVTT", "Kind:", "Language:", "NOTE", "STYLE", "-->", "align")):
                lines.append(line.strip())
    return " ".join(lines)

transcript_text = parse_vtt("subtitle.en.vtt")
print(transcript_text)


00:00:00.040 --> 00:00:01.949 align:start position:0% Lang<00:00:00.320><c> chain</c><00:00:00.560><c> is</c><00:00:00.680><c> a</c><00:00:00.799><c> framework</c><00:00:01.240><c> for</c><00:00:01.480><c> developing</c> 00:00:01.949 --> 00:00:01.959 align:start position:0% Lang chain is a framework for developing 00:00:01.959 --> 00:00:03.909 align:start position:0% Lang chain is a framework for developing applications<00:00:02.720><c> powered</c><00:00:03.040><c> by</c><00:00:03.199><c> large</c><00:00:03.520><c> language</c> 00:00:03.909 --> 00:00:03.919 align:start position:0% applications powered by large language 00:00:03.919 --> 00:00:06.869 align:start position:0% applications powered by large language models<00:00:04.400><c> like</c><00:00:04.920><c> gp4</c><00:00:05.920><c> in</c><00:00:06.080><c> this</c><00:00:06.279><c> course</c><00:00:06.600><c> you</c><00:00:06.720><c> will</c> 00:00:06.869 --> 00:00:06.879 align:start position:0% models like gp4 in this course you will

In [10]:
import re

def parse_vtt_clean(file_path):
    seen = set()
    clean_lines = []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()

            # Skip metadata and timestamps
            if not line or line.startswith(("WEBVTT", "Kind:", "Language:", "NOTE", "STYLE", "align", "-->", "position:", "00:")):
                continue

            # Remove <c> tags and timestamps inside lines
            line = re.sub(r"<[^>]+>", "", line)

            # Remove lines that are duplicates
            if line not in seen:
                seen.add(line)
                clean_lines.append(line)

    return " ".join(clean_lines)

# === Replace this with your actual filename ===
file_path = "lex.en.vtt"
cleaned_text = parse_vtt_clean(file_path)

# Save to a .txt file
with open("cleaned_transcript.txt", "w", encoding="utf-8") as f:
    f.write(cleaned_text)

print("✅ Transcript cleaned and saved to cleaned_transcript.txt")


✅ Transcript cleaned and saved to cleaned_transcript.txt


In [11]:
import re

def parse_vtt_cleaned(file_path):
    cleaned_lines = []
    last_line = None

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()

            # Skip timestamp or metadata lines
            if not line or re.match(r"^\d\d:\d\d:\d\d\.\d+ -->", line):
                continue
            if re.match(r"^(align:start|position|NOTE|STYLE|Kind:|Language:|WEBVTT|n:\d+%)", line):
                continue
            if re.match(r"<\d\d:\d\d:\d\d\.\d+><c>", line):  # skip formatting tags
                continue

            # Remove tags like <c> and <00:00:00.320>
            line = re.sub(r"<.*?>", "", line)

            # Skip duplicate lines
            if line == last_line:
                continue
            last_line = line

            cleaned_lines.append(line)

    return " ".join(cleaned_lines)

# Call the function with your actual file name
transcript = parse_vtt_cleaned("cleaned_transcript.txt")

# Save to a new .txt file
with open("final_transcript.txt", "w", encoding="utf-8") as f:
    f.write(transcript)

print("Cleaned transcript saved to final_transcript.txt")


Cleaned transcript saved to final_transcript.txt


In [12]:
import re

def parse_vtt_cleaned(file_path):
    cleaned_lines = []
    last_line = None

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()

            # Skip empty lines and ones starting with known metadata
            if not line or re.match(r"^(NOTE|STYLE|Kind:|Language:|WEBVTT)", line):
                continue

            # Remove timestamps and metadata anywhere in the line
            line = re.sub(r"\d\d:\d\d:\d\d\.\d+\s-->\s\d\d:\d\d:\d\d\.\d+", "", line)
            line = re.sub(r"\bposition:\d+%|\balign:start|\bn:\d+%", "", line)

            # Remove all <...> tags (e.g. <c>, <00:00:00.000>, etc.)
            line = re.sub(r"<.*?>", "", line)

            # Remove extra whitespace
            line = re.sub(r"\s+", " ", line).strip()

            # Skip if line is same as previous
            if line == last_line or not line:
                continue
            last_line = line

            cleaned_lines.append(line)

    return " ".join(cleaned_lines)

# Run the function and save cleaned transcript
transcript = parse_vtt_cleaned("cleaned_transcript.txt")
with open("final_transcript.txt", "w", encoding="utf-8") as f:
    f.write(transcript)

print("Cleaned transcript saved to final_transcript.txt")


Cleaned transcript saved to final_transcript.txt


In [13]:
print(transcript)

the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down with him for this conversation and i'm sure we will talk many times again in the future this is the lex friedman podcast to support it please check out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal question am i an ai program you wrote to interview people until i get good enough 

In [14]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [15]:
len(chunks)

167

In [16]:
chunks[100]

Document(metadata={}, page_content="clouds where they're gonna go how they're gonna interact when you put two elements together uh and what we try to do is learn a simulation uh uh learner functional that will describe more chemistry types of chemistry so um until now you know you can run expensive simulations but then you can only simulate very small uh molecules very simple molecules we would like to simulate large materials um and so uh today there's no way of doing that and we're building up towards uh building functionals that approximate schrodinger's equation and then allow you to describe uh what the electrons are doing and all materials sort of science and material properties are governed by the electrons and and how they interact so have a good summarization of the simulation through the functional but one that is still close to what the actual simulation would come out with so what um how difficult is that to ask what's involved in that task is it running those those complic

In [17]:
texts = [doc.page_content for doc in chunks]


In [18]:
len(texts)

167

In [19]:
texts[100]

"clouds where they're gonna go how they're gonna interact when you put two elements together uh and what we try to do is learn a simulation uh uh learner functional that will describe more chemistry types of chemistry so um until now you know you can run expensive simulations but then you can only simulate very small uh molecules very simple molecules we would like to simulate large materials um and so uh today there's no way of doing that and we're building up towards uh building functionals that approximate schrodinger's equation and then allow you to describe uh what the electrons are doing and all materials sort of science and material properties are governed by the electrons and and how they interact so have a good summarization of the simulation through the functional but one that is still close to what the actual simulation would come out with so what um how difficult is that to ask what's involved in that task is it running those those complicated simulations yeah and learning"

In [20]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\mohit\AppData\Local\Temp\ipykernel_4296\2379748670.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
d:\langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


embeddings = embedding_model.embed_documents(chunks)
This fails because you're directly passing Document objects to a function that expects plain strings — and it doesn't do the extraction for you.

In [21]:
embeddings = embedding_model.embed_documents(texts)


In [22]:
print(len(embeddings))

167


vector_store = FAISS.from_documents(texts, embedding_model) fails because:
texts is a list of str (strings), not Document objects.

Internally, FAISS.from_documents() tries to access .page_content and .metadata on each item — but str objects don’t have those attributes.

That’s why you get this error:

In [23]:
vector_store = FAISS.from_documents(chunks, embedding_model)


In [24]:
vector_store.index_to_docstore_id

{0: 'b8400f65-68e9-491a-9a5c-30da96e20558',
 1: '71c81c75-2524-4080-95c7-4b1a17e0405d',
 2: '066f8c67-5ef6-41f7-a376-0be1cd145672',
 3: '5f1b0265-028d-422f-9686-42b4ff6a382d',
 4: 'b46f620e-d433-48b9-b6b2-1dd562604c0a',
 5: '5ca4f462-cce7-46a2-a223-28ff806ffd59',
 6: 'f8f13e56-738b-4339-871e-809fe844db5e',
 7: 'e7aead0a-2397-4c74-bbe8-9e1cc795ff08',
 8: 'fb004a70-5227-482a-8d2f-1ef6a73eff7e',
 9: 'dabcb3fe-a2ba-441d-b131-bbdccf7bcb8c',
 10: 'de2a4b62-b73e-4ba4-9f0d-ab5e15036a7e',
 11: 'a997f314-4282-4b02-8556-59a84989fc96',
 12: '8bb86265-f494-4877-be18-9b419f8421fd',
 13: '59fa047c-e305-4f10-9f99-22c9cdefb61b',
 14: 'ee150d91-80c2-4f8b-9b8f-ca7d5ba77775',
 15: '2188c404-ec5a-42c3-b2c0-30e666bc6b71',
 16: 'fa7472ee-dc03-40e5-a564-21c7177bcd23',
 17: 'cec86a4b-b4f4-4f5a-a584-21e83469fba6',
 18: '17221ea2-6e92-4144-aeea-1e158106ca15',
 19: 'c74fbd32-c2a8-449d-8328-2aa9fe544ce5',
 20: 'becd0b21-e8b4-4fb8-b282-7a658a60ccc2',
 21: '9fca1d94-30b2-4dd8-b293-b8ecbbc268fc',
 22: '68122f0a-556a-

In [25]:
vector_store.get_by_ids(['11de5951-29a8-4c03-8a66-2ef6254d4567'])

[]

In [26]:
retriever = vector_store.as_retriever(search_type="similarity",search_kwargs={"k":4})

In [27]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000027149DAA400>, search_kwargs={'k': 4})

In [28]:
retriever.invoke("what is deepmind")

[Document(id='a3b40577-b88b-4439-9ff3-cc3f1a3e1813', metadata={}, page_content="this is tough to uh ask you this question because you probably will say it's everything but let's let's try let's try to think to this because you're in a very interesting position where deepmind is the place of some of the most uh brilliant ideas in the history of ai but it's also a place of brilliant engineering so how much of solving intelligence this big goal for deepmind how much of it is science how much is engineering so how much is the algorithms how much is the data how much is the hardware compute infrastructure how much is it the software computer infrastructure yeah um what else is there how much is the human infrastructure and like just the humans interact in certain kinds of ways in all the space of all those ideas how much does maybe like philosophy how much what's the key if um if if you were to sort of look back like if we go forward 200 years look back what was the key thing that solved in

In [29]:
prompt = PromptTemplate(
  template=""" 
  You are a helpful assistant.
  answer only from the provided transcript context.
  if the context is insufficient, just say you don't know.
  {context}
  Question:{question}
""",
input_variables= ['context','question']
)

In [30]:
question = "is the topic of nuclear fusion discussed in the video? If yes, then what was discussed"
retireved_doc = retriever.invoke(question)

In [31]:
retireved_doc

[Document(id='69ccadc6-cd97-49f6-aa72-3ca8e3d63ac4', metadata={}, page_content="with epfl in switzerland the swiss technical institute who are amazing they have a test reactor that they were willing to let us use which you know i double checked with the team we were going to use carefully and safely i was impressed they managed to persuade them to let us use it and um and it's a it's an amazing test reactor they have there and they try all sorts of pretty crazy experiments on it and um the the the what we tend to look at is if we go into a new domain like fusion what are all the bottleneck problems uh like thinking from first principles you know what are all the bottleneck problems that are still stopping fusion working today and then we look at we you know we get a fusion expert to tell us and then we look at those bottlenecks and we look at the ones which ones are amenable to our ai methods today yes right and and and then and would be interesting from a research perspective from our

In [32]:
context_text = "\n\n".join(doc.page_content for doc in retireved_doc)
context_text

"with epfl in switzerland the swiss technical institute who are amazing they have a test reactor that they were willing to let us use which you know i double checked with the team we were going to use carefully and safely i was impressed they managed to persuade them to let us use it and um and it's a it's an amazing test reactor they have there and they try all sorts of pretty crazy experiments on it and um the the the what we tend to look at is if we go into a new domain like fusion what are all the bottleneck problems uh like thinking from first principles you know what are all the bottleneck problems that are still stopping fusion working today and then we look at we you know we get a fusion expert to tell us and then we look at those bottlenecks and we look at the ones which ones are amenable to our ai methods today yes right and and and then and would be interesting from a research perspective from our point of view from an ai point of view and that would address one of their\n\n

In [33]:
final_prompt = prompt.invoke({"context":context_text,"question":question})

In [34]:
final_prompt

StringPromptValue(text=" \n  You are a helpful assistant.\n  answer only from the provided transcript context.\n  if the context is insufficient, just say you don't know.\n  with epfl in switzerland the swiss technical institute who are amazing they have a test reactor that they were willing to let us use which you know i double checked with the team we were going to use carefully and safely i was impressed they managed to persuade them to let us use it and um and it's a it's an amazing test reactor they have there and they try all sorts of pretty crazy experiments on it and um the the the what we tend to look at is if we go into a new domain like fusion what are all the bottleneck problems uh like thinking from first principles you know what are all the bottleneck problems that are still stopping fusion working today and then we look at we you know we get a fusion expert to tell us and then we look at those bottlenecks and we look at the ones which ones are amenable to our ai methods 

In [35]:
llm = Ollama(model="deepseek-r1:1.5b",temperature = 0.1)


In [36]:
answer = llm.invoke(final_prompt)

In [37]:
print(answer)

<think>
Okay, I need to figure out if the topic of nuclear fusion is covered in the provided transcript context. Let me read through the text again.

The assistant starts by talking about EPFL in Switzerland and their test reactor, which is a big deal because it's an amazing project. They mention that they were willing to let people use it for research. The assistant then goes into details about experiments on the reactor, like controlling plasma shapes and containing it for long periods. 

The assistant discusses how fusion has several challenges: physics material science engineering, especially in building and containing the plasma. They talk about droplets and other shapes used in fusion reactors. There's also a mention of a controller that can handle different shapes to manage energy production.

Then, the assistant talks about collaboration with experts from EPFL, like a paper titled "Pushing the frontiers of density functionals by solving the fractional electron problem." This se

In [38]:
llm1 = Ollama(model="gemma3:1b",temperature = 0.1)


In [39]:
answer = llm1.invoke(final_prompt)
print(answer)

Yes, the topic of nuclear fusion is discussed in the video.

The video discusses the following aspects of nuclear fusion:

*   **Bottleneck problems:** The video explores the challenges hindering fusion research, including thinking from first principles, physics, material science, and engineering.
*   **Fusion expert collaboration:** It highlights the collaboration with fusion experts, specifically Epfl in Switzerland, who have a test reactor and are working on solutions to these challenges.
*   **Plasma shaping and control:** The video details the creation of plasma shapes and the use of a controller to maintain the plasma's structure and containment for extended periods.


#building a chain

In [40]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [41]:
def format_doc(retireved_doc):
  context_text = "\n\n".join(doc.page_content for doc in retireved_doc)
  return context_text

In [42]:
parallel_chain = RunnableParallel({
  'context':retriever | RunnableLambda(format_doc),
  'question':RunnablePassthrough()
})

In [43]:
parallel_chain.invoke("who is demis")

{'context': "the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down with him for this conversation and i'm sure we will talk many times again in the future this is the lex friedman podcast to support it please check out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal question am i an ai program you wrote to interview people until i get

In [44]:
parser = StrOutputParser()

In [45]:
main_chain = parallel_chain | prompt | llm | parser # using deepseek

In [46]:
main_chain1 = parallel_chain | prompt | llm1 | parser #using gemma

In [47]:
main_chain1.invoke("can you summarize the key points discussed in the video")

"Here’s a summary of the key points discussed in the video:\n\n*   **The Goal:** The video focuses on the pursuit of artificial intelligence, specifically aiming to understand the totality of what could be understood.\n*   **Three Trees/Branches:** The conversation traces the evolution of AI's goals, starting with simple game-breaking systems and progressing to building general learning systems.\n*   **Protein Folding:** The video highlights the significant progress made in protein folding, particularly through the AlphaFold project, and the immense amount of data and engineering involved.\n*   **Data and Innovation:** The video emphasizes that cracking complex problems like protein folding required a massive amount of data and innovative algorithms.\n*   **Fundamental Physics:** The discussion touches on the idea that certain things might not be understandable through current AI because they relate to fundamental physics, like the simulation hypothesis.\n*   **Human Understanding:** T

In [48]:
main_chain.invoke("can you summarize the video")

"<think>\nOkay, so I need to figure out how to answer this question. The user provided a transcript context where someone is talking about an explanation of physics concepts, starting from fundamental explanations and moving into more detailed theories like consciousness, life, and gravity. They also mention that it's an honor to pause the discussion to talk about these topics and then leave with some words from Edskar Dykstra about computer science being no more than astronomy.\n\nNow, the user is asking me to summarize the video based on this transcript context. But wait, I don't have access to the actual video content because it's not provided in the transcript. So, how can I summarize a video that hasn't been shown here?\n\nI remember that when someone asks for a summary of a video without the transcript, they might be looking for an explanation based on what's typically covered in such discussions. Since the context mentions fundamental physics concepts and then delves into consci

In [49]:
#from langchain.memory.buffer import ConversationBufferMemory


In [50]:
#conversation_memory = ConversationBufferMemory(memory_key="chat_history",max_len=50, return_messages=True,)